In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import pandas as pd
from sklearn import datasets
# from sklearn.model_selection import train_test_split
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.stats.diagnostic as diag
from sklearn.linear_model import LogisticRegression
#Metrics
from sklearn.metrics import make_scorer, accuracy_score,precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score,recall_score,f1_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
import cProfile

In [2]:
# Seleccionar solo las variables numéricas relevantes
datos = pd.read_csv("train.csv", encoding="latin1")
numericas = datos.select_dtypes(include='number')
numericas = numericas.drop(["Fireplaces", "GarageYrBlt", "Id", "MSSubClass", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "MoSold", "YrSold"], axis=1)

numericas = numericas.dropna()

In [3]:
# Crear variable respuesta que clasifica las casas en Económicas, Intermedias o Caras
terciles = np.percentile(numericas["SalePrice"], [33.33, 66.67])
limite_economicas = terciles[0]
limite_caras = terciles[1]
numericas["Clase"] = pd.cut(numericas["SalePrice"], bins=[0, limite_economicas, limite_caras, float("inf")], labels=["Económicas", "Intermedias", "Caras"])
dummies = pd.get_dummies(numericas['Clase'])
datos = pd.concat([numericas,dummies], axis=1)
economica = datos.pop("Económicas") 
intermedias = datos.pop("Intermedias")
caras = datos.pop("Caras")
datos.pop("Clase")
X = datos
y = caras
random.seed(42)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [5]:
# Importar la biblioteca SVM
from sklearn.svm import SVC

# Crear un objeto SVM con un kernel lineal
svm = SVC(kernel='linear')

# Entrenar el modelo con los datos de entrenamiento
svm.fit(X_train, y_train)

# Predecir los valores de la variable respuesta para los datos de prueba
y_pred = svm.predict(X_test)

# Evaluar el rendimiento del modelo utilizando diferentes métricas de evaluación
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Precisión:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("Puntuación F1:", f1_score(y_test, y_pred, average='weighted'))

Exactitud: 1.0
Precisión: 1.0
Recall: 1.0
Puntuación F1: 1.0


In [6]:
# Importar GridSearchCV
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros a evaluar
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'degree': [2, 3, 4], 'gamma': ['scale', 'auto']}

# Crear un objeto GridSearchCV con un SVM
svm = SVC()
grid_search = GridSearchCV(svm, param_grid=param_grid, cv=5)

# Entrenar el objeto GridSearchCV con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros: ", grid_search.best_params_)

# Predecir los valores de la variable respuesta para los datos de prueba utilizando el mejor modelo
y_pred = grid_search.predict(X_test)

# Evaluar el rendimiento del modelo utilizando diferentes métricas de evaluación
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Precisión:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("Puntuación F1:", f1_score(y_test, y_pred, average='weighted'))


Mejores parámetros:  {'C': 0.1, 'degree': 4, 'gamma': 'auto', 'kernel': 'poly'}
Exactitud: 1.0
Precisión: 1.0
Recall: 1.0
Puntuación F1: 1.0


In [7]:
# Importar GridSearchCV
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros a evaluar
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'degree': [2, 3, 4], 'gamma': ['scale', 'auto']}

# Crear un objeto GridSearchCV con un SVM
svm = SVC()
grid_search = GridSearchCV(svm, param_grid=param_grid, cv=5)

# Entrenar el objeto GridSearchCV con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros: ", grid_search.best_params_)

# Predecir los valores de la variable respuesta para los datos de prueba utilizando el mejor modelo
y_pred = grid_search.predict(X_test)

# Evaluar el rendimiento del modelo utilizando diferentes métricas de evaluación
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Precisión:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("Puntuación F1:", f1_score(y_test, y_pred, average='weighted'))


Mejores parámetros:  {'C': 0.1, 'degree': 4, 'gamma': 'auto', 'kernel': 'poly'}
Exactitud: 1.0
Precisión: 1.0
Recall: 1.0
Puntuación F1: 1.0


In [8]:
matrix = confusion_matrix(y_test, y_pred)

# 9. Compare la eficiencia del mejor modelo de SVM con los resultados obtenidos en los algoritmos de las hojas de trabajo anteriores que usen la misma variable respuesta

In [9]:
X_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [10]:
def entrenar_arbol(X_train, y_train, X_test, y_test):
    # Crear modelo de árbol de clasificación
    arbol = DecisionTreeClassifier(random_state=0)
    # Entrenar modelo
    arbol.fit(X_train, y_train)
    # Predecir valores para conjunto de prueba
    y_pred = arbol.predict(X_test)
    # Evaluar precisión del modelo
    precision = accuracy_score(y_test, y_pred)
    # Predecir etiquetas para conjunto de prueba
    y_pred = arbol.predict(X_test)
    # Calcular precisión del modelo
    precision = accuracy_score(y_test, y_pred)
    # Creamos la matriz de confusion
    cm = confusion_matrix(y_test, y_pred)
    # Calculamos algunas métricas de evaluación
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    # Imprimimos las métricas de evaluación
    print("Exactitud: {:.2f}".format(accuracy))
    print("Precisión: {:.2f}".format(precision))
    print("Sensibilidad: {:.2f}".format(recall))
    print("Puntuación F1: {:.2f}".format(f1))

In [11]:
def random_forest_model(X_train, y_train, X_test, y_test, n_estimators=100, max_depth=None):
    # Crear el modelo de Random Forest con los parámetros deseados
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    # Entrenar el modelo con los datos de entrenamiento
    rf.fit(X_train, y_train)   
    # Predecir las clases para los datos de prueba
    y_pred = rf.predict(X_test)
    # Calcular precisión del modelo
    precision = accuracy_score(y_test, y_pred)
    # Creamos la matriz de confusion
    cm = confusion_matrix(y_test, y_pred)
    # Calculamos algunas métricas de evaluación
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    # Imprimimos las métricas de evaluación
    print("Exactitud: {:.2f}".format(accuracy))
    print("Precisión: {:.2f}".format(precision))
    print("Sensibilidad: {:.2f}".format(recall))
    print("Puntuación F1: {:.2f}".format(f1))

In [12]:
entrenar_arbol(X_train, y_train, X_test, y_test)

Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Puntuación F1: 1.00


In [13]:
cProfile.run('entrenar_arbol(X_train, y_train, X_test, y_test)')

Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Puntuación F1: 1.00
         19460 function calls (19199 primitive calls) in 0.013 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.013    0.013 3843723223.py:1(entrenar_arbol)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
       17    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(any)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argmax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_1d)
       12    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(average)
        9    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
       11    0.000    0.000    0.000    0.000 <__array_

In [14]:
random_forest_model(X_train, y_train, X_test, y_test)

Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Puntuación F1: 1.00


In [15]:
cProfile.run('random_forest_model(X_train, y_train, X_test, y_test)')

Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Puntuación F1: 1.00
         290525 function calls (287609 primitive calls) in 0.227 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.227    0.227 3932651776.py:1(random_forest_model)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
      100    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(amax)
      116    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(any)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argmax)
      102    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_1d)
       11    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(average)
      109    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(bincount)
      209    0.000    0.000    0.001    0.000 <_

In [16]:
print("El mejor modelo para predecir, aunque los modelos tengan una \
exactitud del 1.00, es el Arbol de decision. Esto se debe a que \
este arbol muestra un tiempo de ejecucion menor al resto, demorandose \
0.013 segundos. El modelo que mas tardo fue el random forest, con un \
tiempo de 0.231 segundos.")

El mejor modelo para predecir, aunque los modelos tengan una exactitud del 1.00, es el Arbol de decision. Esto se debe a que este arbol muestra un tiempo de ejecucion menor al resto, demorandose 0.013 segundos. El modelo que mas tardo fue el random forest, con un tiempo de 0.231 segundos.


# 10 & 11. Genere un modelo de regresión, use para esto la variable del precio de la casa.

In [17]:
X = datos
y = caras

random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [18]:
logReg = LogisticRegression(solver='liblinear')
logReg.fit(X_train,y_train)
y_pred = logReg.predict(X_test)
cm = confusion_matrix(y_test,y_pred)

In [19]:
logReg = LogisticRegression(solver='liblinear')
logReg.fit(X_train,y_train)
y_pred = logReg.predict(X_test)
cm = confusion_matrix(y_test,y_pred)

In [20]:
accuracy=accuracy_score(y_test,y_pred)
precision =precision_score(y_test, y_pred,average='micro')
recall =  recall_score(y_test, y_pred,average='micro')
f1 = f1_score(y_test,y_pred,average='micro')
print('Matriz de confusión para detectar si es cara o no\n',cm)
print('Accuracy: ',accuracy)
print('Precision: ', precision)
print('Recall score:', recall)
print('F1 score: ', f1)

Matriz de confusión para detectar si es cara o no
 [[166   1]
 [  0  72]]
Accuracy:  0.99581589958159
Precision:  0.99581589958159
Recall score: 0.99581589958159
F1 score:  0.99581589958159


In [21]:
k_folds = KFold(n_splits = 5, shuffle=True, random_state=42)

scores = cross_val_score(logReg, X, y, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9874477 0.9958159 0.9916318 0.9958159 0.9916318]
Average CV Score:  0.992468619246862
Number of CV Scores used in Average:  5


Dado que se obtuvo un buen puntaje para la precisión del modelo \
y además se tuvo un alto porcentaje promedio para el modelo, se puede \
determinar que este es un modelo bueno para determinar si una casa \
llegará a ser cara o no.
La regresión logística se utiliza cuando la variable dependiente \
es categórica (binaria o multinomial) y la relación entre las variables \
independientes y dependientes no es lineal. La regresión lineal se utiliza cuando la variable dependiente es continua y la relación entre las variables independientes y dependientes es lineal. La mejor regresión para este caso es la logística, ya que se utiliza comúnmente con variables categóricas en problemas de clasificación, como predecir si un cliente comprará un producto o no en función de su información.

# 12. Informe de datos

GridSearchCV se utiliza para la sintonización de hiperparámetros en 
modelos de aprendizaje automático. Al entrenar un modelo de aprendizaje 
automático, necesitamos establecer varios hiperparámetros, que controlan 
el comportamiento del modelo durante el entrenamiento. Estos 
hiperparámetros no se aprenden a partir de los datos, sino que 
son establecidos por el usuario antes de entrenar el modelo. La elección 
de los hiperparámetros puede afectar mucho el rendimiento del modelo. 
Toma un diccionario de hiperparámetros y sus posibles valores, y 
luego realiza una búsqueda en cuadrícula sobre todas las posibles 
combinaciones de hiperparámetros. Para cada combinación de hiperparámetros, 
entrena y evalúa el modelo utilizando validación cruzada y devuelve 
la mejor combinación de hiperparámetros que resultó en el mejor rendimiento.
Podemos ver que GSCV tuvo una eficiencia excelente: 
Exactitud: 1.0
Precisión: 1.0
Recall: 1.0
Puntuación F1: 1.0
Sin embargo, hay que recordar que tal exactitud (y demás parámetros) de 1.00
pueden significar un sobreajuste a los datos, y que el modelo no se pueda
ajustar muy bien a los nuevos datos.